<a href="https://colab.research.google.com/github/satishgc227/Derby-project/blob/main/Logistic_Regression%2CXGB%2CRandom_Forest%2CNaive_bayes%2CSVC%2CDeep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from sklearn.decomposition import PCA

from sklearn.svm import SVC
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
import time

from sklearn.preprocessing import MaxAbsScaler
from sklearn.naive_bayes import GaussianNB

from sklearn.neighbors import KNeighborsClassifier

import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout



In [2]:
# Load in the data
df = pd.read_csv('/content/edited complete.csv',encoding='latin',nrows=100000)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


One hot encoding:

In [3]:
#Unique values of the columns
df['race_type\xa0'].unique()
df['course_type'].unique()
df['track_condition\xa0'].unique()
df['track_id'].unique()


array(['AQU', 'BEL', 'SAR'], dtype=object)

In [4]:
s=df[['race_type\xa0','course_type','track_condition\xa0','track_id']]

In [5]:
ohe=OneHotEncoder()
feature_array=ohe.fit_transform(df[['race_type\xa0','course_type','track_condition\xa0','track_id']]).toarray()

In [6]:
feature_labels=ohe.categories_
print(feature_labels)

[array(['ALW', 'AOC', 'CLM', 'SST', 'STK', nan], dtype=object), array(['D', 'I', 'M', 'O', 'T', nan], dtype=object), array(['FM ', 'FT ', 'GD ', 'MY ', 'SY ', 'YL ', nan], dtype=object), array(['AQU', 'BEL', 'SAR'], dtype=object)]


In [7]:
np.hstack(feature_labels)

array(['ALW', 'AOC', 'CLM', 'SST', 'STK', nan, 'D', 'I', 'M', 'O', 'T',
       nan, 'FM ', 'FT ', 'GD ', 'MY ', 'SY ', 'YL ', nan, 'AQU', 'BEL',
       'SAR'], dtype=object)

In [8]:
feature_labels=np.hstack(feature_labels)
f=pd.DataFrame(feature_array,columns=feature_labels)
pd.concat([df,f],axis=1)

,track_id,race_date,race_number,program_number,trakus_index,latitude,longitude,distance_id,course_type,track_condition,...,FM,FT,GD,MY,SY,YL,NaN,AQU,BEL,SAR
0,AQU,01-01-2019,9,6,73,40.672946,-73.827587,600.0,D,GD,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,AQU,01-01-2019,9,6,74,40.672990,-73.827568,600.0,D,GD,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,AQU,01-01-2019,9,6,63,40.672510,-73.827781,600.0,D,GD,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,AQU,01-01-2019,9,6,64,40.672553,-73.827762,600.0,D,GD,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,AQU,01-01-2019,9,6,65,40.672596,-73.827742,600.0,D,GD,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40923,AQU,05-01-2019,5,3,29,40.670348,-73.828764,650.0,D,SY,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
40924,AQU,05-01-2019,5,3,198,40.674965,-73.830345,650.0,D,SY,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
40925,AQU,05-01-2019,5,3,199,40.674929,-73.830370,650.0,D,SY,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
40926,AQU,05-01-2019,5,3,200,40.674894,-73.830394,650.0,D,SY,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [9]:

df.drop(['race_date','track_id','course_type','track_condition\xa0','race_type\xa0','jockey\xa0','program_number\xa0'],axis=1,inplace=True)
df=pd.concat([df,f],axis=1)

In [10]:
df.isnull().any()


race_number            False
trakus_index           False
latitude               False
longitude              False
distance_id             True
run_up_distance         True
purse                   True
post_time               True
weight_carried          True
odds                    True
position_at_finish      True
ALW                    False
AOC                    False
CLM                    False
SST                    False
STK                    False
NaN                    False
D                      False
I                      False
M                      False
O                      False
T                      False
NaN                    False
FM                     False
FT                     False
GD                     False
MY                     False
SY                     False
YL                     False
NaN                    False
AQU                    False
BEL                    False
SAR                    False
dtype: bool

In [11]:
#Removing the null values

df.replace([np.inf,-np.inf],np.nan ,inplace=True)
df.dropna(inplace=True)
df.isnull().any()

race_number            False
trakus_index           False
latitude               False
longitude              False
distance_id            False
run_up_distance        False
purse                  False
post_time              False
weight_carried         False
odds                   False
position_at_finish     False
ALW                    False
AOC                    False
CLM                    False
SST                    False
STK                    False
NaN                    False
D                      False
I                      False
M                      False
O                      False
T                      False
NaN                    False
FM                     False
FT                     False
GD                     False
MY                     False
SY                     False
YL                     False
NaN                    False
AQU                    False
BEL                    False
SAR                    False
dtype: bool

In [12]:
!pip install movecolumn
import movecolumn as mc
mc.MoveToLast(df,'position_at_finish\xa0')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for movecolumn: filename=movecolumn-0.0.7-py3-none-any.whl size=10440 sha256=18f1389178dc93ca17d07442ec66f652543999636a7738f87943d05bea9fe849
  Stored in directory: /root/.cache/pip/wheels/4c/ad/b7/671cd4a6e1464bb816aea3bacdcc933acf25e3d47946a7edcf
Successfully built movecolumn


,race_number,trakus_index,latitude,longitude,distance_id,run_up_distance,purse,post_time,weight_carried,odds,...,FT,GD,MY,SY,YL,NaN,AQU,BEL,SAR,position_at_finish
0,9,73,40.672946,-73.827587,600.0,48.0,25000.0,420.0,120.0,2090.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0
1,9,74,40.672990,-73.827568,600.0,48.0,25000.0,420.0,120.0,2090.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0
2,9,63,40.672510,-73.827781,600.0,48.0,25000.0,420.0,120.0,2090.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0
3,9,64,40.672553,-73.827762,600.0,48.0,25000.0,420.0,120.0,2090.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0
4,9,65,40.672596,-73.827742,600.0,48.0,25000.0,420.0,120.0,2090.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40922,5,28,40.670306,-73.828783,650.0,32.0,75000.0,226.0,119.0,1240.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,6.0
40923,5,29,40.670348,-73.828764,650.0,32.0,75000.0,226.0,119.0,1240.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,6.0
40924,5,198,40.674965,-73.830345,650.0,32.0,75000.0,226.0,119.0,1240.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,6.0
40925,5,199,40.674929,-73.830370,650.0,32.0,75000.0,226.0,119.0,1240.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,6.0


In [13]:
X = df.iloc[:,:-1]
y = df.iloc[:, -1]

Principal Component Analysis

In [14]:
pca = PCA(n_components=10)
X = pca.fit_transform(X)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['float', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [15]:
X.shape
X

array([[-4.10006814e+04,  6.41844391e+02, -2.68924738e+02, ...,
         3.69243064e+00, -2.52709361e-01,  1.52202692e-01],
       [-4.10006812e+04,  6.41837670e+02, -2.68727728e+02, ...,
         3.69235469e+00, -2.52690960e-01,  1.52286485e-01],
       [-4.10006834e+04,  6.41911601e+02, -2.70894841e+02, ...,
         3.69319018e+00, -2.52887490e-01,  1.51366458e-01],
       ...,
       [ 8.99099461e+03, -5.94449078e+02, -4.54670471e+02, ...,
        -4.15092457e-02, -3.85200932e-01,  1.51570982e-01],
       [ 8.99099481e+03, -5.94455799e+02, -4.54473461e+02, ...,
        -4.15861908e-02, -3.85249314e-01,  1.51632773e-01],
       [ 8.99099501e+03, -5.94462520e+02, -4.54276450e+02, ...,
        -4.16631409e-02, -3.85297871e-01,  1.51694578e-01]])

Training and testing the data

In [16]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size= 0.2, random_state = 1)


Standardising the X

In [17]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [18]:
print('X_train dimension= ', X_train.shape)
print('X_test dimension= ', X_test.shape)
print('y_train dimension= ', y_train.shape)
print('y_train dimension= ', y_test.shape)

X_train dimension=  (32741, 10)
X_test dimension=  (8186, 10)
y_train dimension=  (32741,)
y_train dimension=  (8186,)


Logistic Regression 

In [19]:
#fitting the model
lm = linear_model.LogisticRegression(multi_class='ovr', solver='liblinear')
lm.fit(X_train, y_train)

LogisticRegression(multi_class='ovr', solver='liblinear')

In [20]:
#Predictions
y_pred=lm.predict(X_test)

In [21]:
#Classification report,accuracy score
result1 = classification_report(y_test, y_pred)
print(result1)
result2 = accuracy_score(y_test,y_pred)
print(result2)

              precision    recall  f1-score   support

         1.0       0.50      0.73      0.59       763
         2.0       0.53      0.24      0.33      1090
         3.0       0.61      0.64      0.62       990
         4.0       0.55      0.45      0.49       824
         5.0       0.51      0.46      0.48      1364
         6.0       0.33      0.36      0.34      1129
         7.0       0.57      0.75      0.65       745
         8.0       0.50      0.64      0.56       706
         9.0       0.49      0.63      0.55       367
        10.0       0.00      0.00      0.00       176
        11.0       1.00      1.00      1.00        32

    accuracy                           0.50      8186
   macro avg       0.51      0.54      0.51      8186
weighted avg       0.50      0.50      0.49      8186

0.5047642316149523


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


XGB Classifier:

In [22]:
#Input the parameters
xgb = XGBClassifier(n_estimators=10)






In [23]:
#training time
training_start = time.perf_counter()
xgb.fit(X_train, y_train)
training_end = time.perf_counter()

In [24]:
#predicting time
prediction_start = time.perf_counter()
preds1 = xgb.predict(X_test)
prediction_end = time.perf_counter()

In [25]:
xgb_train_time = training_end-training_start
xgb_prediction_time = prediction_end-prediction_start

In [26]:

print("Time consumed for training: %4.3f" % (xgb_train_time))
print("Time consumed for prediction: %6.5f seconds" % (xgb_prediction_time))

Time consumed for training: 2.588
Time consumed for prediction: 0.03055 seconds


In [27]:
accuracy_score(y_test,preds1)

0.948204251160518

Random Forest classifier

In [28]:
#Input the parameters
rfc = RandomForestClassifier(n_estimators=100)


In [29]:
#Training time
training_start = time.perf_counter()
rfc.fit(X_train, y_train)
training_end = time.perf_counter()

In [30]:
#predicting time
prediction_start = time.perf_counter()
preds2 = rfc.predict(X_test)
prediction_end = time.perf_counter()

In [31]:
rfc_train_time = training_end-training_start
rfc_prediction_time = prediction_end-prediction_start

In [32]:

print("Time consumed for training: %4.3f seconds" % (rfc_train_time))
print("Time consumed for prediction: %6.5f seconds" % (rfc_prediction_time))

Time consumed for training: 6.198 seconds
Time consumed for prediction: 0.13587 seconds


In [33]:
accuracy_score(y_test,preds2)

1.0

Support Vector Machine

In [34]:
#Input the parameters
svc = SVC()


In [35]:
#Training time
training_start = time.perf_counter()
svc.fit(X_train, y_train)
training_end = time.perf_counter()

In [36]:
#Testing time
prediction_start = time.perf_counter()
preds3 = svc.predict(X_test)
prediction_end = time.perf_counter()

In [37]:

svc_train_time = training_end-training_start
svc_prediction_time = prediction_end-prediction_start

In [38]:

print("Time consumed for training: %4.3f seconds" % (svc_train_time))
print("Time consumed for prediction: %6.5f seconds" % (svc_prediction_time))

Time consumed for training: 14.480 seconds
Time consumed for prediction: 3.96199 seconds


In [39]:
print(accuracy_score(y_test,preds3)) 

0.9926704129000733


Naive Bayes

In [57]:
#Input the parameters 
gnb = GaussianNB()


In [58]:
#Training time
training_start = time.perf_counter()
gnb.fit(X_train, y_train)
training_end = time.perf_counter()

In [59]:
#Predicting time
prediction_start = time.perf_counter()
preds4 = gnb.predict(X_test)
prediction_end = time.perf_counter()

In [61]:
gnb_train_time = training_end-training_start
gnb_prediction_time = prediction_end-prediction_start

In [62]:
print("Time consumed for training: %4.3f seconds" % (gnb_train_time))
print("Time consumed for prediction: %6.5f seconds" % (gnb_prediction_time))

Time consumed for training: 0.017 seconds
Time consumed for prediction: 0.01263 seconds


In [63]:
accuracy_score(y_test,preds4)

0.4166870266308331

KNeighbours Classifier:

In [47]:
#Input the parameters
knn = KNeighborsClassifier()

In [48]:
#Training time
training_start = time.perf_counter()
knn.fit(X_train, y_train)
training_end = time.perf_counter()

In [49]:
#Predicting Time
prediction_start = time.perf_counter()
preds5 = knn.predict(X_test)
prediction_end = time.perf_counter()

In [50]:
knn_train_time = training_end-training_start
knn_prediction_time = prediction_end-prediction_start

In [51]:
print("Time consumed for training: %4.3f seconds" % (knn_train_time))
print("Time consumed for prediction: %6.5f seconds" % (knn_prediction_time))

Time consumed for training: 0.056 seconds
Time consumed for prediction: 0.22081 seconds


In [52]:
print(accuracy_score(y_test,preds5))

1.0


Deep learning 

In [53]:
#Initialising the model
model=Sequential()
model.add(Dense(10,activation='relu',input_dim=10)) #input layer
model.add(Dropout(0.1))
model.add(Dense(10,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                110       
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dropout_1 (Dropout)         (None, 10)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 231
Trainable params: 231
Non-trainable params: 0
_________________________________________________________________


In [70]:
#Applying optimizers and callbacks
optimizer=keras.optimizers.Adam(learning_rate=0.0001)
callback=EarlyStopping(monitor='val_loss', min_delta=0.0001,
    patience=5,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False)

In [71]:
#Compiling the model
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [72]:
#Fitting the model
history=model.fit(X_train,y_train,batch_size=200, epochs=100,
                    validation_data = (X_test,y_test), verbose=1, callbacks=callback)

Epoch 1/100
164/164 [==============================] - 1s 4ms/step - loss: -5439427584.0000 - accuracy: 0.0852 - val_loss: -5366087680.0000 - val_accuracy: 0.0932
Epoch 2/100
164/164 [==============================] - 1s 4ms/step - loss: -5459422208.0000 - accuracy: 0.0852 - val_loss: -5366277632.0000 - val_accuracy: 0.0932
Epoch 3/100
164/164 [==============================] - 1s 4ms/step - loss: -5448906240.0000 - accuracy: 0.0852 - val_loss: -5366467072.0000 - val_accuracy: 0.0932
Epoch 4/100
164/164 [==============================] - 1s 4ms/step - loss: -5455844864.0000 - accuracy: 0.0852 - val_loss: -5366656512.0000 - val_accuracy: 0.0932
Epoch 5/100
164/164 [==============================] - 1s 4ms/step - loss: -5453266432.0000 - accuracy: 0.0852 - val_loss: -5366845952.0000 - val_accuracy: 0.0932
Epoch 6/100
164/164 [==============================] - 1s 4ms/step - loss: -5445197312.0000 - accuracy: 0.0852 - val_loss: -5367035392.0000 - val_accuracy: 0.0932
Epoch 7/100
164/164 [=